In [ ]:
!pip install requests_html

In [ ]:
import asyncio
from requests_html import AsyncHTMLSession
import pandas as pd
import re

In [ ]:
url = "https://www.amazon.com/product-reviews/B09MV3KMPT"

In [10]:
async def extract_reviews(url, page_limit=None, page_number = 1):
    session = AsyncHTMLSession()
    reviews_extracted = 0
        
    page_url = f"{url}/ref=cm_cr_arp_d_paging_btm_next_{page_number + 1}?ie=UTF8&reviewerType=all_reviews&pageNumber={page_number + 1}"
    r = await session.get(page_url)
    await r.html.arender()
    reviews = r.html.find("div[data-hook='review']")

    df = pd.DataFrame(columns = columns)
    for review in reviews:
      # Convert the dictionary to a DataFrame and concatenate it to the original DataFrame
      new_data = pd.DataFrame([extract_review_data(review)])
      df = pd.concat([df, new_data], ignore_index=True)
      reviews_extracted += 1

    await session.close()
    # print(df)
    # print(f"Total reviews extracted: {reviews_extracted}")
    return df

In [18]:
data = pd.DataFrame(columns= columns)

for i in range(1):
  data = pd.concat([data, await extract_reviews(url=url, page_number=i)], ignore_index= True)

[INFO] Starting Chromium download.
100%|███████████████████████████████████████████████████████████████████████████████| 137M/137M [00:04<00:00, 34.0Mb/s]
[INFO] Beginning extraction
[INFO] Chromium extracted to: C:\Users\sohyeon\AppData\Local\pyppeteer\pyppeteer\local-chromium\588429


Lydia Martinez
4.0 out of 5 stars Yes x3
Reviewed in the United States on February 20, 2023
Color: BlackVerified Purchase
I was so skeptical at the time i opened it n saw such a small suction. But i used it in my car where my dog always rides shot gun. N the car is full of fur. I vacuumed the seat the floor the dash the console the door. HECK I VACUUMED THE DOG. i wanted to see how good it was. It vacuumed his loose hair right off. I had a cup full of his fur.
The reviews were right down side was the brush. Not that it didnt work but u want ant extra one or a bigger one. I didnt even use the exta accesseries. But i was just amazed on how much fur it picked up. N i keep it in my car still plugged in so when ever im driving n im at a red light i vacuum myself the fur thats on me. I look pretty silly at the red light. But its getting att the fur.
46 people found this helpful
Helpful
Report abuse
Showing 0 comments
There was a problem loading comments right now. Please try again later.
Ohu

Jacquelyn B.
VINE VOICE
4.0 out of 5 stars nice accessories for a car, but I prefer a more general dust buster
Reviewed in the United States on January 18, 2023
Color: BlackVerified Purchase
This is a cute and moderately effective vacuum with a lot of pluses for cleaning a car: the storage bag is great, the included accessories are ideal for reaching between seats, the suction is strong enough to get the one cashew your kid has somehow wedged into a cranny. It's easy enough to clean out after use, up to and including changing the filter. And it's small enough to travel in the car full time without sacrificing much cargo space.

The downside of a car specific vacuum is that it is specific to cars. This is designed to plug into a car outlet, so you can't easily clean a spill on the couch if all you can find is your car vacuum. This one is a bit small for cleaning the floor of the car, and the size somewhat limits the suction.
43 people found this helpful
Helpful
Report abuse
Showing 0 co

cynthia perez
3.0 out of 5 stars Good for what it's worth...suction could be better
Reviewed in the United States on January 16, 2023
Color: BlackVerified Purchase
I received this as a Xmas gift, which I really appreciated. My wife knows I like keeping our cars as clean as possible. My review in brief: nothing will come close to the vacuum cleaners available at car washes or vacuums like a Shop Vac, but I do wish the This Worx vacuum cleaner had better suction. It picks up top layer (loose) debris fairly easily, but it takes a bit more elbow grease to pick up lint and small particles that get into the fibers of the carpet. This a nice on-the-go, compact car vacuum option for what it is.

While the manufacturer recommends having the car on to get maximum power out of the ThisWorx, I didn't feel or see much difference with the car off (plus who wants to leave their car running for 10-15 minutes). For those hard to suction particles, I recommend using the brush that's included and really 

In [19]:
print(data)

             Reviewer Name Star Rating               Date       Location  \
0           Lydia Martinez         4.0  February 20, 2023  United States   
1                   Ohukno         4.0      June 18, 2021  United States   
2                Nicole N.         4.0      March 3, 2023  United States   
3  Jacquelyn B. VINE VOICE         4.0   January 18, 2023  United States   
4           Liz Curruchich         4.0   January 16, 2023  United States   
5                    AndoD         4.0   February 3, 2023  United States   
6                 Brenda S         4.0     March 14, 2023  United States   
7             Animal Crazy         4.0     March 11, 2023  United States   
8            cynthia perez         3.0   January 16, 2023  United States   
9           Leslie Morales         4.0     March 13, 2023  United States   

                                               Title  \
0                                             Yes x3   
1                        Good device with some flaw

In [20]:
data.shape

(10, 6)

In [21]:
data.to_csv("vacuum_reviews.csv", index=False)

In [15]:
def extract_review_data(review):

  print(review.text)
  # Split the review text into lines
  lines = review.text.split('\n')

  # Determine if the reviewer has a tag (e.g., VINE VOICE)
  has_tag = not lines[2].startswith('Reviewed')

  # Extract the required information
  reviewer_name = lines[0] if not has_tag else ' '.join(lines[:2])
  #review_title = lines[1 if not has_tag else 2].split(' ', 1)[1].strip()
  #review_title = re.split(r'\d+(\.\d+)? out of 5 stars', lines[1 if not has_tag else 2])[1].strip()
  review_title = re.sub(r'^\d+(\.\d+)? out of 5 stars ', '', lines[1 if not has_tag else 2]).strip()
  star_rating = float(re.search(r"\d+\.\d+", lines[1 if not has_tag else 2]).group())
  date_string = re.search(r"on\s(.+)$", lines[2 if not has_tag else 3]).group(1)
  location = re.search(r"Reviewed in the (.+?)\s{1,2}on", lines[2 if not has_tag else 3]).group(1)


  # Find the line with "people found this helpful" and extract the review content up to that line
  # helpful_line_index = [i for i, line in enumerate(lines) if "people found this helpful" in line][0]
  # review_content = "\n".join(lines[4 if not has_tag else 5:helpful_line_index+1]).strip()

  review_content = []
  for line in lines[4:]:
    if "found this helpful" in line or "Report abuse" in line or "Showing 0 comments" in line or "Helpful" in line or lines[0].strip() in line:
      break
    review_content.append(line.strip())
  review_content = "\n".join(review_content)

  # Create a dictionary with the extracted information
  review_data = {
      'Reviewer Name': reviewer_name,
      'Star Rating': star_rating,
      'Date': date_string,
      'Location': location,
      'Title': review_title,
      'Review Content': review_content
  }

  # Convert the dictionary to a pandas DataFrame
  # df = pd.DataFrame([review_data])
  # print(df)

  return review_data

In [17]:
columns = ["Reviewer Name", "Star Rating", "Date", "Location", "Title", "Review Content"]